\# Worksheet 08

Name:  Gauravdeep Singh Bindra
UID: U23346660

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review


Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [3]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

c1 = np.random.normal(mean, stdev, 10).tolist()
print(c1)

[4.018710911495575, 4.535465384077881, 6.534159911651646, 5.907882156846552, 5.526348937457065, 5.774580649925459, 4.600195058953441, 3.3824136111571232, 6.56012980221487, 5.316846218056069]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [4]:
mean = 8
stdev = 1
c2 = np.random.normal( mean ,stdev, 10 ).tolist()
print(c2)

[9.264959771673091, 10.082726462331886, 9.074814816678643, 9.204041123575838, 8.110020645619104, 8.451104793783365, 9.882084294106875, 7.166878432262782, 7.966886901953358, 8.232308563450378]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [5]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = c1[-1]
        n1 = len(c1)
        c1 = c1[:n1-1]
        data.append(p1)
    else:
        p2 = c2[-1]
        n2 = len(c2)
        c2 = c2[:n2-1]
        data.append(p2)
print(data)

[8.232308563450378, 5.316846218056069, 7.966886901953358, 7.166878432262782, 6.56012980221487, 9.882084294106875, 3.3824136111571232, 4.600195058953441, 8.451104793783365, 8.110020645619104]


In [6]:
print(len(c1))

6


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

Parameters 
1st distribution - Mean 5, Variance 1
2nd distribution - Mean 8, Variance 1
Probability of distributions = 1/2 

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(C_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(C_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [7]:
np.array(data).reshape(-1, 1)

array([[8.23230856],
       [5.31684622],
       [7.9668869 ],
       [7.16687843],
       [6.5601298 ],
       [9.88208429],
       [3.38241361],
       [4.60019506],
       [8.45110479],
       [8.11002065]])

In [8]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

c1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(c1)
c2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(c2)

prob_c = [ len(c1) / (len(c1) + len(c2)) , len(c2)/(len(c1) + len(c2))]
mean = [ sum(c1)/len(c1) , sum(c2)/len(c2)]
var = [ sum(map(lambda x : (x - mean[0])**2, c1)) / len(c1) , sum(map(lambda x : (x - mean[1])**2, c2)) / len(c2) ]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[5.316846218056069, 3.3824136111571232, 4.600195058953441]
[8.232308563450378, 7.966886901953358, 7.166878432262782, 6.56012980221487, 9.882084294106875, 8.451104793783365, 8.110020645619104]
P(C_1) = 0.3,  P(C_2) = 0.7
mean_1 = 4.4331516293888775,  mean_2 = 8.052773347627246
var_1 = 0.6376233387859875,  var_2 = 0.9372468090050542


/usr/local/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [9]:
from scipy.stats import norm
norm.pdf(data[0], mean[0], var[0])

1.2226567635835334e-08

e) For each data point, compute `P(C_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [10]:
from scipy.stats import norm

prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", round(pdf_i[j],5))
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]
    # print('prob_x',prob_x)
    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append( (prob_c[0] * pdf_i[0])/prob_x )
    prob_c1_x.append( (prob_c[1] * pdf_i[1])/prob_x )
    # print('prob_c0_x',prob_c0_x)


probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(round(p[1],100)))
    print("Probability of coming from C_2 = " + str(round(p[2],100)))
    print()


point =  8.232308563450378
probability of observing that point if it came from cluster 0 =  0.0
probability of observing that point if it came from cluster 1 =  0.41792
point =  5.316846218056069
probability of observing that point if it came from cluster 0 =  0.23947
probability of observing that point if it came from cluster 1 =  0.00601
point =  7.966886901953358
probability of observing that point if it came from cluster 0 =  0.0
probability of observing that point if it came from cluster 1 =  0.42387
point =  7.166878432262782
probability of observing that point if it came from cluster 0 =  6e-05
probability of observing that point if it came from cluster 1 =  0.2723
point =  6.56012980221487
probability of observing that point if it came from cluster 0 =  0.0024
probability of observing that point if it came from cluster 1 =  0.11976
point =  9.882084294106875
probability of observing that point if it came from cluster 0 =  0.0
probability of observing that point if it came from 

f) Having computed `P(C_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(C_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [12]:
prob_c = [sum(prob_c0_x)/ len(prob_c0_x), sum(prob_c1_x)/ len(prob_c1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_c0_x, data)]) / sum(prob_c0_x), sum([x[0] * x[1] for x in zip(prob_c1_x, data)]) / sum(prob_c1_x) ]
var = [ sum([x[0] * (x[1]-mean[0])**2 for x in zip(prob_c0_x, data)]) / sum(prob_c0_x) , sum([x[0] * (x[1]-mean[1])**2 for x in zip(prob_c1_x, data)]) / sum(prob_c1_x)]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))


P(C_1) = 0.2951435187596801,  P(C_2) = 0.7048564812403199
mean_1 = 4.422726233864407,  mean_2 = 8.032199461399223
var_1 = 0.646651206233956,  var_2 = 0.9896019237425809


These values are a bit different, not a lot. There is around a 1% differencer in probabilities which gets to a 5% difference in variance. /

g) Update `P(C_j | X_i)`. Comment on any differences or lack thereof you observe.

In [13]:
prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", round(pdf_i[j],5))
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]
    # print('prob_x',prob_x)
    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append( (prob_c[0] * pdf_i[0])/prob_x )
    prob_c1_x.append( (prob_c[1] * pdf_i[1])/prob_x )
    # print('prob_c0_x',prob_c0_x)


probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(round(p[1],100)))
    print("Probability of coming from C_2 = " + str(round(p[2],100)))
    print()

point =  8.232308563450378
probability of observing that point if it came from cluster 0 =  0.0
probability of observing that point if it came from cluster 1 =  0.39498
point =  5.316846218056069
probability of observing that point if it came from cluster 0 =  0.23719
probability of observing that point if it came from cluster 1 =  0.00934
point =  7.966886901953358
probability of observing that point if it came from cluster 0 =  0.0
probability of observing that point if it came from cluster 1 =  0.40226
point =  7.166878432262782
probability of observing that point if it came from cluster 0 =  8e-05
probability of observing that point if it came from cluster 1 =  0.27505
point =  6.56012980221487
probability of observing that point if it came from cluster 0 =  0.00262
probability of observing that point if it came from cluster 1 =  0.13334
point =  9.882084294106875
probability of observing that point if it came from cluster 0 =  0.0
probability of observing that point if it came fro

All probabilities have been updated 

h) Use `P(C_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [14]:
prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", round(pdf_i[j],5))
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]
    # print('prob_x',prob_x)
    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append( (prob_c[0] * pdf_i[0])/prob_x )
    prob_c1_x.append( (prob_c[1] * pdf_i[1])/prob_x )
    # print('prob_c0_x',prob_c0_x)


probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(round(p[1],1)))
    print("Probability of coming from C_2 = " + str(round(p[2],1)))
    print()

point =  8.232308563450378
probability of observing that point if it came from cluster 0 =  0.0
probability of observing that point if it came from cluster 1 =  0.39498
point =  5.316846218056069
probability of observing that point if it came from cluster 0 =  0.23719
probability of observing that point if it came from cluster 1 =  0.00934
point =  7.966886901953358
probability of observing that point if it came from cluster 0 =  0.0
probability of observing that point if it came from cluster 1 =  0.40226
point =  7.166878432262782
probability of observing that point if it came from cluster 0 =  8e-05
probability of observing that point if it came from cluster 1 =  0.27505
point =  6.56012980221487
probability of observing that point if it came from cluster 0 =  0.00262
probability of observing that point if it came from cluster 1 =  0.13334
point =  9.882084294106875
probability of observing that point if it came from cluster 0 =  0.0
probability of observing that point if it came fro

Creating a hard assignment by rounding to 1 decimal place. 

### Clustering Aggregation

| Point | C | P |
|-------|---|---|
| A     | 0 | a |
| B     | 0 | b |
| C     | 2 | b |
| D     | 1 | c |
| E     | 1 | d |

a) Fill in the following table where for each pair of points determine whether C and P agree or disagree on how to cluster that pair.

| Pair | Disagreement |
|------|--------------|
| A  B |      Disagree      |
| A  C |      Disagree       |
| A  D |      Disagree       |
| A  E |      Disagree       |
| B  C |      Disagree       |
| B  D |      Disagree       |
| B  E |      Disagree       |
| C  D |      Disagree       |
| C  E |      Disagree       |
| D  E |      Disagree       |


As datasets become very large, this process can become computationally challenging.

b) Given N points, what is the formula for the number of unique pairs of points one can create?

nC2 = n!/((n-2)!*2!) = n(n-1)/2

Assume that clustering C clusters all points in the same cluster and clustering P clusters points as such:

| Point | P |
|-------|---|
| A     | 0 |
| B     | 0 |
| C     | 0 |
| D     | 1 |
| E     | 1 |
| F     | 2 |
| G     | 2 |
| H     | 2 |
| I     | 2 |

c) What is the maximum number of disagreements there could be for a dataset of this size? (use the formula from b)?

9*8/2 = 36 disagreements, though in this case as there can be 6 agreements so 30 disagreemnets b/w C and P 

d) If we look at cluster 0. There are (3 x 2) / 2 = 3 pairs that agree with C (since all points in C are in the same cluster). For each cluster, determine how many agreements there are. How many total agreements are there? How many disagreements does that mean there are between C and P?

Cluster 0 = 3 agreements, Cluster 1 = 1 agreement, Cluster 2 = 6 agreements. Total agreeements can be = 6 since we can only match 1 cluster from P to the all-element cluster in C

e) Assuming that filtering the dataset by cluster number is a computationally easy operation, describe an algorithm inspired by the above process that can efficiently compute disagreement distances on large datasets.

Input:
Two clustering results: A, B
Dataset: D
Output:
Disagreement distance: dis_dist
Steps:
Initialize Counter:
Initialize a counter disagreement_count to 0.

Iterate through Data Points:
For each data point p in D, perform the following sub-steps:

a. Identify Clusters:
Identify the clusters C_A and C_B to which point p belongs in clustering A and B, respectively.

b. Filter Neighboring Points:
Retrieve all points P_A in cluster C_A and P_B in cluster C_B.

c. Compute Local Disagreement:
Compute the local disagreement by checking whether points that are in the same cluster in A are in different clusters in B and vice versa. Specifically:

Increment disagreement_count for each pair (p_i, p_j) where p_i ∈ P_A and p_j ∈ P_A but p_i and p_j are not in the same cluster in B.
Increment disagreement_count for each pair (p_x, p_y) where p_x ∈ P_B and p_y ∈ P_B but p_x and p_y are not in the same cluster in A.
(Note: Avoid double-counting the pairs in different iterations)